In [1]:
from pprint import pprint
import json

with open('bias-study-export.json') as f:
    data = json.load(f)['logs']
    
with open('images/_airlines.json') as f:
    airlines = json.load(f)

with open('images/_states.json') as f:
    states = json.load(f)

pprint(data['82534293'])

{'approximateVis_airline': {'HowMany': {'answer': '4000',
                                        'confidence': 'agree',
                                        'type': 'HowMany'},
                            'HowManyCompare_EV': {'answer': '2000',
                                                  'confidence': 'weak_agree',
                                                  'data': 'EV',
                                                  'type': 'HowManyCompare'},
                            'HowManyCompare_VX': {'answer': '3500',
                                                  'confidence': 'neutral',
                                                  'data': 'VX',
                                                  'type': 'HowManyCompare'}},
 'approximateVis_states': {'HowMany': {'answer': '6000',
                                       'confidence': 'strong_agree',
                                       'type': 'HowMany'},
                           'HowManyCompare_MA': {'answer': '30

In [2]:
from collections import OrderedDict 
from copy import deepcopy

rows = []

for p in data.items():
    d = OrderedDict({
        'id': p[0]
    })
    
    item = p[1]
    
    d['userAgent'] = item['userAgent']
    d['isMobile'] = item['isMobile']
    d['done'] = 'done' in item
    
    cond = item['cond']
    
    errorAirline = cond['errorAirline']
    seqAirline = cond['seqAirline']
    errorStates = cond['errorStates']
    seqStates = cond['seqStates']
    focusAirline = cond['focusAirline']
    focusState = cond['focusState']
    
    for k, v in cond.items():
        d[k] = v
        
    howManyIndex = 0
    for k, v in item['approximateVis_airline'].items():
        t = v['type']
        
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'approx.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
    
    howManyIndex = 0
    for k, v in item['approximateVis_states'].items():
        t = v['type']
        
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'approx.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
     
    def jaccard_similarity(x,y):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    
    # precise
            
    howManyIndex = 0
    for k, v in item['preciseVis_airline'].items():
        t = v['type']
        
        precise = airlines["{}_{}".format(1, 1)]
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'precise.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'precise'] = precise[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorAirline > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            
    howManyIndex = 0
    for k, v in item['preciseVis_states'].items():
        t = v['type']
        
        precise = states["{}_{}".format(1, 1)]
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'precise.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'precise'] = precise[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorStates > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
    
    for k, v in item['timing'].items():
         d['timing.' + k] = v
            
    for k, v in item['demographics'].items():
         d['demographics.' + k] = v
    
    rows.append(d)

pprint(rows)

[OrderedDict([('id', '57003634'),
              ('userAgent',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) '
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 '
               'Safari/537.36'),
              ('isMobile', False),
              ('done', True),
              ('focusAirline', 'DL'),
              ('errorAirline', 500),
              ('secondCondition', 'states'),
              ('seqStates', 14),
              ('firstCondition', 'airline'),
              ('errorStates', 1000),
              ('studyCondition', 'both'),
              ('seqAirline', 10),
              ('focusState', 'MA'),
              ('approx.airline.howManyCompare.0.answer', 1000),
              ('approx.airline.howManyCompare.0.data', 'WN'),
              ('approx.airline.howManyCompare.0.approx', 1500),
              ('approx.airline.howManyCompare.0.confidence', 'strong_disagree'),
              ('approx.airline.howManyCompare.1.answer', 2000),
              (

In [3]:
import csv

with open('bias-study-export.csv', 'w') as f:
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in rows:
        writer.writerow(row)

        
import pandas as pd

pd.read_csv("bias-study-export.csv")

,id,userAgent,isMobile,done,focusAirline,errorAirline,secondCondition,seqStates,firstCondition,errorStates,...,timing.approximate_first_html,timing.approximate_second_html,timing.index_html,timing.instructions_html,demographics.troubles,demographics.familiar,demographics.age,demographics.gender,demographics.improved,demographics.problems
0,57003634,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,False,True,DL,500,states,14,airline,1000,...,48561,32497,7427,59095,no,expert,26,female,Separate the two comparisons of precise & appr...,Small image sizes
1,82534293,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1...,False,True,AA,1,airline,11,states,1000,...,40055,145279,8695,26016,no,expert,26,male,nope,nope
